In [69]:
import requests
import os
import pandas as pd

In [72]:
class AccuWeatherApiClient:
    """
    Client for getting data from AccuWeather API.
    """
    def __init__(self, api_key: str):
        self.base_url = "https://dataservice.accuweather.com"
        if api_key is None:
            raise Exception("API key cannot be set to None.")
        self.api_key = api_key
        
    def get_current_weather(
        self, location_key: int
    ) -> dict:
        """Extract location data from a city name using the location API
        
        Args: 
            city_key: city key in text to search for
            
        Returns:
            A dict with the current weather metrics 
            
        Raises:
            Exception when it is not possible to extract data from the API.
        """
        location_url = f"{self.base_url}/currentconditions/v1/{location_key}"
        params = {
            "apikey" : self.api_key,
            "language" : "en-us",
            "details" : "true",
        }
        response = requests.get(url=location_url, params=params)
        if response.status_code == 200:
            return response.json()[0]
        else:
            raise Exception(
                f"Failes to extract data from AccuWeather API. Status Code: {response.status_code}. Response: {response.text}"
            )
    def get_location(self, location_name: str) -> str:
        """Extract location data from a city name using the location API
        
        Args: 
            q: city text to search for.
            
        Returns:
            A string with the key of the city
            
        Raises:
            Exception when it is not possible to extract data from the API.
        """
        location_url = f"{self.base_url}/locations/v1/cities/search"
        params = {
            "q": location_name,
            "apikey" : self.api_key,
            "language" : "en-us",
            "details" : "false",
        }
        response = requests.get(url=location_url, params=params)
        if response.status_code == 200 and response.json()[0].get("Key") is not None:
            return response.json()[0].get("Key")
        else:
            raise Exception(
                f"Failes to extract data from AccuWeather API. Status Code: {response.status_code}. Response: {response.text}"
            )

In [101]:
def extract_current_weather(
    accuweather_client: AccuWeatherApiClient,
    location_name: str
) -> pd.DataFrame:
    """
    Extract forecast data from AccuWeather API.

    The forecast data is extracted from a desired location based on the location_key.
    
    The forecast_days argument define the number of forecast days returned. Possible
    values are 1, 5, 10, and 15.
    """
    location_key = accuweather_client.get_location(
        city_name = location_name
    )
    
    data = accuweather_client.get_current_weather(
        location_key = location_key
    )
    data['location_name'] = location_name
    data['location_key'] = location_key
    return pd.json_normalize(data=data)

In [102]:
accuweather_client = AccuWeatherApiClient(api_key="5Kzw2rB1gGZ5YJN0cGFgGjjYMaUz1PYP")
response = extract_current_weather(accuweather_client = accuweather_client, location_name = 'pucon')
type(response)

pandas.core.frame.DataFrame

In [98]:
response.city_name

0    pucon
Name: city_name, dtype: object

In [46]:
response.get('WeatherText')

'Mostly sunny'

In [51]:
response.get('Temperature').get('Metric').get('Value')

25.3

In [53]:
response.get('RealFeelTemperature').get('Metric').get('Value')

30.3

In [56]:
response.get('RealFeelTemperature').get('Metric').get('Phrase')

'Very Warm'

In [61]:
response.get('Wind').get('Speed').get('Metric').get('Value')

9.4

In [65]:
response.get('Wind').get('Direction').get('Localized')

'WNW'

AttributeError: 'AccuWeatherApiClient' object has no attribute 'get_location'